In [1]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline



Multiclass margin classifier
============================

.. meta::
    :property="og:description": Using PyTorch to implement a multiclass
        quantum variational classifier on MNIST data.
    :property="og:image": https://pennylane.ai/qml/_images/margin_2.png

.. related::

   tutorial_variational_classifier Variational quantum classifier
   tutorial_data_reuploading_classifier Data-reuploading classifier

*Author: PennyLane dev team. Posted: 9 Apr 2020. Last updated: 28 Jan 2021.*

In this tutorial, we show how to use the PyTorch interface for PennyLane
to implement a multiclass variational classifier. We consider the iris database
from UCI, which has 4 features and 3 classes. We use multiple one-vs-all
classifiers with a margin loss (see `Multiclass Linear SVM
<http://cs231n.github.io/linear-classify/>`__) to classify data. Each classifier is implemented
on an individual variational circuit, whose architecture is inspired by
`Farhi and Neven (2018) <https://arxiv.org/abs/1802.06002>`__ as well as
`Schuld et al. (2018) <https://arxiv.org/abs/1804.00633>`__.

|

.. figure:: ../demonstrations/multiclass_classification/margin_2.png
    :align: center
    :width: 50%
    :target: javascript:void(0)

|


Initial Setup
~~~~~~~~~~~~~

We import PennyLane, the PennyLane-provided version of NumPy,
relevant torch modules, and define the constants that will
be used in this tutorial.

Our feature size is 4, and we will use amplitude embedding.
This means that each possible amplitude (in the computational basis) will
correspond to a single feature. With 2 qubits (wires), there are
4 possible states, and as such, we can encode a feature vector
of size 4.


In [3]:
import pennylane as qml
import torch
from pennylane import numpy as np
from torch.autograd import Variable
import torch.optim as optim

np.random.seed(0)
torch.manual_seed(0)

num_classes = 3
margin = 0.15
feature_size = 3
batch_size = 10
lr_adam = 0.01
train_split = 0.75
# the number of the required qubits is calculated from the number of features
num_qubits = int(np.ceil(np.log2(feature_size)))
num_layers = 6
total_iterations = 100

dev = qml.device("default.qubit", wires=num_qubits)

Quantum Circuit
~~~~~~~~~~~~~~~

We first create the layer that will be repeated in our variational quantum
circuits. It consists of rotation gates for each qubit, followed by
entangling/CNOT gates



In [4]:
def layer(W):
    for i in range(num_qubits):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
    for j in range(num_qubits - 1):
        qml.CNOT(wires=[j, j + 1])
    if num_qubits >= 2:
        # Apply additional CNOT to entangle the last with the first qubit
        qml.CNOT(wires=[num_qubits - 1, 0])

We now define the quantum nodes that will be used. As we are implementing our
multiclass classifier as multiple one-vs-all classifiers, we will use 3 QNodes,
each representing one such classifier. That is, ``circuit1`` classifies if a
sample belongs to class 1 or not, and so on. The circuit architecture for all
nodes are the same. We use the PyTorch interface for the QNodes.
Data is embedded in each circuit using amplitude embedding.
<div class="alert alert-info"><h4>Note</h4><p>For demonstration purposes we are using a very simple circuit here. 
    You may find that other choices, for example more 
    elaborate measurements, increase  the power of the classifier.</p></div>



In [5]:
def circuit(weights, feat=None):
    qml.templates.embeddings.AmplitudeEmbedding(feat, range(num_qubits), pad=0.0, normalize=True)
    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))


qnodes = []
for iq in range(num_classes):
    qnode = qml.QNode(circuit, dev, interface="torch")
    qnodes.append(qnode)

The variational quantum circuit is parametrized by the weights. We use a
classical bias term that is applied after processing the quantum circuit's
output. Both variational circuit weights and classical bias term are optimized.



In [6]:
def variational_classifier(q_circuit, params, feat):
    weights = params[0]
    bias = params[1]
    return q_circuit(weights, feat=feat) + bias

Loss Function
~~~~~~~~~~~~~

Implementing multiclass classifiers as a number of one-vs-all classifiers
generally evokes using the margin loss. The output of the $i$ th classifier, $c_i$
on input $x$ is interpreted as a score, $s_i$ between [-1,1].
More concretely, we have:

\begin{align}s_i = c_i(x; \theta)\end{align}

The multiclass margin loss attempts to ensure that the score for the correct
class is higher than that of incorrect classes by some margin. For a sample $(x,y)$
where $y$ denotes the class label, we can analytically express the mutliclass
loss on this sample as:

\begin{align}L(x,y) = \sum_{j \ne y}{\max{\left(0, s_j - s_y + \Delta)\right)}}\end{align}

where $\Delta$ denotes the margin. The margin parameter is chosen as a hyperparameter.
For more information, see `Multiclass Linear SVM <http://cs231n.github.io/linear-classify/>`__.



In [7]:
def multiclass_svm_loss(q_circuits, all_params, feature_vecs, true_labels):
    loss = 0
    num_samples = len(true_labels)
    for i, feature_vec in enumerate(feature_vecs):
        # Compute the score given to this sample by the classifier corresponding to the
        # true label. So for a true label of 1, get the score computed by classifer 1,
        # which distinguishes between "class 1" or "not class 1".
        s_true = variational_classifier(
            q_circuits[int(true_labels[i])],
            (all_params[0][int(true_labels[i])], all_params[1][int(true_labels[i])]),
            feature_vec,
        )
        s_true = s_true.float()
        li = 0

        # Get the scores computed for this sample by the other classifiers
        for j in range(num_classes):
            if j != int(true_labels[i]):
                s_j = variational_classifier(
                    q_circuits[j], (all_params[0][j], all_params[1][j]), feature_vec
                )
                s_j = s_j.float()
                li += torch.max(torch.zeros(1).float(), s_j - s_true + margin)
        loss += li

    return loss / num_samples

Classification Function
~~~~~~~~~~~~~~~~~~~~~~~

Next, we use the learned models to classify our samples. For a given sample,
compute the score given to it by classifier $i$, which quantifies how likely it is that
this sample belongs to class $i$. For each sample, return the class with the highest score.



In [8]:
def classify(q_circuits, all_params, feature_vecs, labels):
    predicted_labels = []
    for i, feature_vec in enumerate(feature_vecs):
        scores = np.zeros(num_classes)
        for c in range(num_classes):
            score = variational_classifier(
                q_circuits[c], (all_params[0][c], all_params[1][c]), feature_vec
            )
            scores[c] = float(score)
        pred_class = np.argmax(scores)
        predicted_labels.append(pred_class)
    return predicted_labels


def accuracy(labels, hard_predictions):
    loss = 0
    for l, p in zip(labels, hard_predictions):
        if torch.abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / labels.shape[0]
    return loss

Data Loading and Processing
~~~~~~~~~~~~~~~~~~~~~~~~~~~

Now we load in the iris dataset and normalize the features so that the sum of the feature
elements squared is 1 ($\ell_2$ norm is 1).



In [11]:
data = np.loadtxt("data/multiclass_classification/iris.csv", delimiter=",")
X = torch.tensor(data[:, 0:feature_size])
print("First X sample, original  :", X[0])

# normalize each input
normalization = torch.sqrt(torch.sum(X ** 2, dim=1))
X_norm = X / normalization.reshape(len(X), 1)
print("First X sample, normalized:", X_norm[0])
X_norm

First X sample, original  : tensor([5.1000, 3.5000, 1.4000], dtype=torch.float64)
First X sample, normalized: tensor([0.8042, 0.5519, 0.2208], dtype=torch.float64)


tensor([[0.8042, 0.5519, 0.2208],
        [0.8286, 0.5073, 0.2367],
        [0.8058, 0.5486, 0.2229],
        [0.8005, 0.5395, 0.2610],
        [0.7914, 0.5698, 0.2216],
        [0.7855, 0.5673, 0.2473],
        [0.7811, 0.5774, 0.2377],
        [0.8026, 0.5458, 0.2408],
        [0.8070, 0.5319, 0.2568],
        [0.8181, 0.5176, 0.2505],
        [0.8041, 0.5510, 0.2234],
        [0.7874, 0.5578, 0.2625],
        [0.8232, 0.5145, 0.2401],
        [0.8027, 0.5600, 0.2053],
        [0.8115, 0.5597, 0.1679],
        [0.7750, 0.5982, 0.2039],
        [0.7957, 0.5746, 0.1915],
        [0.8042, 0.5519, 0.2208],
        [0.8076, 0.5384, 0.2409],
        [0.7805, 0.5815, 0.2296],
        [0.8177, 0.5149, 0.2574],
        [0.7874, 0.5713, 0.2316],
        [0.7762, 0.6075, 0.1687],
        [0.8085, 0.5232, 0.2695],
        [0.7765, 0.5500, 0.3074],
        [0.8269, 0.4962, 0.2646],
        [0.7994, 0.5436, 0.2558],
        [0.8068, 0.5430, 0.2327],
        [0.8165, 0.5339, 0.2198],
        [0.795

In [62]:
import matplotlib.pyplot as plt
import csv

data = []
count = 1
index = 1
with open('activity.csv', 'r') as file:
# with open('tranquil.csv', 'r') as file:
# with open('piano.csv', 'r') as file:
# with open('active.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        count += 1
        if count % 2 == 0 and count > 2:
            x = [int(float(row[6])), int(float(row[7])), int(float(row[8]))]
            data.append(x / nump.linalg.norm(x))
#             data.append(int(row[5]))
#             data.append(np.array([int(float(row[6])), int(float(row[7])), int(float(row[8]))]))
            

print(np.shape(data))

data_2 = []
count = 1
index = 1
with open('tranquil.csv', 'r') as file:
# with open('tranquil.csv', 'r') as file:
# with open('piano.csv', 'r') as file:
# with open('active.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        count += 1
        if count % 2 == 0 and count > 2:
            x = [int(float(row[6])), int(float(row[7])), int(float(row[8]))]
            data_2.append(x / nump.linalg.norm(x))
#             data_2.append(np.array([int(float(row[6])), int(float(row[7])), int(float(row[8]))]))

print(np.shape(data_2))

data_3 = []
count = 1
index = 1
with open('working.csv', 'r') as file:
# with open('tranquil.csv', 'r') as file:
# with open('piano.csv', 'r') as file:
# with open('active.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        count += 1
        if count % 2 == 0 and count > 2:
            x = [int(float(row[6])), int(float(row[7])), int(float(row[8]))]
            data_3.append(x / nump.linalg.norm(x))
#             data_3.append([int(float(row[6])), int(float(row[7])), int(float(row[8]))])

print(np.shape(data_3))

#### next step is to normalize
# x = data[10]
x = [int(float(row[6])), int(float(row[7])), int(float(row[8]))]
data_3.append(x / np.linalg.norm(x))

(16179, 3)
(14308, 3)
(15288, 3)


In [72]:
data_3[0]

array([0.51256251, 0.68888267, 0.51256251])

In [82]:
X = []
Y = []

# X = np.append([data, data_2, data_3])
for i in range(len(data)):
    X.append(np.array(data[i]))

for i in range(len(data_2)):
    X.append(np.array(data_2[i]))

#     X = np.append(data_2[i], X)

for i in range(len(data_3)):
    X.append(np.array(data_3[i]))

#     X = np.append(data_3[i], X)

# X = np.array(X, requires_grad=False)
# print(X)

# X = np.append([data, data_2, data_3])
labels = 0*np.ones(len(data), dtype=np.int8).numpy()
for i in range(len(labels)):
    Y.append(labels[i])

labels_2 = 1*np.ones(len(data_2), dtype=np.int8)
for i in range(len(labels_2)):
    Y.append(labels_2[i])

#     X = np.append(data_2[i], X)

labels_3 = 2*np.ones(len(data_3), dtype=np.int8)
for i in range(len(data_3)):
    Y.append(labels_3[i])
    
print(type(X))
X = torch.tensor(np.array(X))
Y = torch.tensor(Y)
# print(Y)
X
    


<class 'list'>


tensor([[0.5126, 0.6889, 0.5126],
        [0.7852, 0.6106, 0.1028],
        [0.9534, 0.2968, 0.0550],
        ...,
        [0.8452, 0.4742, 0.2466],
        [0.8461, 0.4621, 0.2658],
        [0.8461, 0.4621, 0.2658]], dtype=torch.float64)

In [83]:
X

tensor([[0.5126, 0.6889, 0.5126],
        [0.7852, 0.6106, 0.1028],
        [0.9534, 0.2968, 0.0550],
        ...,
        [0.8452, 0.4742, 0.2466],
        [0.8461, 0.4621, 0.2658],
        [0.8461, 0.4621, 0.2658]], dtype=torch.float64)

In [84]:
# print("First X sample, original  :", X_sample[0])
print("First X sample, normalized:", X[0])

First X sample, normalized: tensor([0.5126, 0.6889, 0.5126], dtype=torch.float64)


In [85]:
data = np.loadtxt("data/multiclass_classification/iris.csv", delimiter=",")
Y_data = torch.tensor(data[:, -1])
print(type(X_sample))
X_sample = torch.tensor(data[:, 0:feature_size])
print("First X sample, original  :", X_sample[0])

# normalize each input
normalization = torch.sqrt(torch.sum(X_sample ** 2, dim=1))
X_norm_smaple = X_sample / normalization.reshape(len(X_sample), 1)
print("First X sample, normalized:", X_norm[0])
print("First X sample, normalized:", np.shape(X_norm))

print("First X sample, normalized:", Y_data[0])

<class 'torch.Tensor'>
First X sample, original  : tensor([5.1000, 3.5000, 1.4000], dtype=torch.float64)
First X sample, normalized: tensor([0.8042, 0.5519, 0.2208], dtype=torch.float64)
First X sample, normalized: torch.Size([150, 3])
First X sample, normalized: tensor(0., dtype=torch.float64)


In [86]:
def load_and_process_data():
#     data = np.loadtxt("data/multiclass_classification/iris.csv", delimiter=",")
#     X = torch.tensor(data[:, 0:feature_size])
#     print("First X sample, original  :", X[0])

#     # normalize each input
#     normalization = torch.sqrt(torch.sum(X ** 2, dim=1))
#     X_norm = X / normalization.reshape(len(X), 1)
#     print("First X sample, normalized:", X_norm[0])
    load()
    
#     Y = torch.tensor(data[:, -1])
    return X, Y


# Create a train and test split.
def split_data(feature_vecs, Y):
    num_data = len(Y)
    num_train = int(train_split * num_data)
    index = np.random.permutation(range(num_data))
    print(index)
    print(feature_vecs)
    feat_vecs_train = feature_vecs[index[:num_train]]
    Y_train = Y[index[:num_train]]
    feat_vecs_test = feature_vecs[index[num_train:]]
    Y_test = Y[index[num_train:]]
    return feat_vecs_train, feat_vecs_test, Y_train, Y_test

Training Procedure
~~~~~~~~~~~~~~~~~~

In the training procedure, we begin by first initializing randomly the parameters
we wish to learn (variational circuit weights and classical bias). As these are
the variables we wish to optimize, we set the ``requires_grad`` flag to ``True``. We use
minibatch training---the average loss for a batch of samples is computed, and the
optimization step is based on this. Total training time with the default parameters
is roughly 15 minutes.



In [ ]:
def training(features, Y):
    num_data = Y.shape[0]
    feat_vecs_train, feat_vecs_test, Y_train, Y_test = split_data(features, Y)
    num_train = Y_train.shape[0]
    q_circuits = qnodes

    # Initialize the parameters
    all_weights = [
        Variable(0.1 * torch.randn(num_layers, num_qubits, 3), requires_grad=True)
        for i in range(num_classes)
    ]
    all_bias = [torch.tensor(0.1 * torch.ones(1), requires_grad=True) for i in range(num_classes)]
    optimizer = optim.Adam(all_weights + all_bias, lr=lr_adam)
    params = (all_weights, all_bias)
#     print(params)
    print("Num params: ", 3 * num_layers * num_qubits * 3 + 3)

    costs, train_acc, test_acc = [], [], []

    # train the variational classifier
    for it in range(total_iterations):
        batch_index = np.random.randint(0, num_train, (batch_size,))
#         print(feat_vecs_train[0])
#         print(batch_index)
#         print(feat_vecs_train[0])
#         feat_vecs_train_np = np.array(feat_vecs_train)
#         Y_train_np = np.array(Y_train)
        
        feat_vecs_train_batch = []
        for i in range(len(batch_index)):
            feat_vecs_train_batch.append(feat_vecs_train[i])
#         = feat_vecs_train_np[batch_index]
        
        Y_train_batch = []
        for i in range(len(batch_index)):
            Y_train_batch.append(Y_train[i])
            
        print('DATA_PREPARED')
#         Y_train_batch = Y_train_np[batch_index]

        optimizer.zero_grad()
        curr_cost = multiclass_svm_loss(q_circuits, params, feat_vecs_train_batch, Y_train_batch)
        curr_cost.backward()
        optimizer.step()

        # Compute predictions on train and validation set
        predictions_train = classify(q_circuits, params, feat_vecs_train, Y_train)
        predictions_test = classify(q_circuits, params, feat_vecs_test, Y_test)
        acc_train = accuracy(Y_train, predictions_train)
        acc_test = accuracy(Y_test, predictions_test)

        print(
            "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc test: {:0.7f} "
            "".format(it + 1, curr_cost.item(), acc_train, acc_test)
        )

        costs.append(curr_cost.item())
        train_acc.append(acc_train)
        test_acc.append(acc_test)

    return costs, train_acc, test_acc


# We now run our training algorithm and plot the results. Note that
# for plotting, the matplotlib library is required

# features, Y = load_and_process_data()
# load()
import numpy as nump

# Y_array = nump.array(Y)

costs, train_acc, test_acc = training(X, Y)

import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()
iters = np.arange(0, total_iterations, 1)
colors = ["tab:red", "tab:blue"]
ax1.set_xlabel("Iteration", fontsize=17)
ax1.set_ylabel("Cost", fontsize=17, color=colors[0])
ax1.plot(iters, costs, color=colors[0], linewidth=4)
ax1.tick_params(axis="y", labelsize=14, labelcolor=colors[0])

ax2 = ax1.twinx()
ax2.set_ylabel("Test Acc.", fontsize=17, color=colors[1])
ax2.plot(iters, test_acc, color=colors[1], linewidth=4)

ax2.tick_params(axis="x", labelsize=14)
ax2.tick_params(axis="y", labelsize=14, labelcolor=colors[1])

plt.grid(False)
plt.tight_layout()
plt.show()

[34944 20588 24356 ... 13798 12853 11981]
tensor([[0.5126, 0.6889, 0.5126],
        [0.7852, 0.6106, 0.1028],
        [0.9534, 0.2968, 0.0550],
        ...,
        [0.8452, 0.4742, 0.2466],
        [0.8461, 0.4621, 0.2658],
        [0.8461, 0.4621, 0.2658]], dtype=torch.float64)
Num params:  111
DATA_PREPARED


<ipython-input-100-65bb42d7de6d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_bias = [torch.tensor(0.1 * torch.ones(1), requires_grad=True) for i in range(num_classes)]


Iter:     1 | Cost: 0.2208004 | Acc train: 0.4550274 | Acc test: 0.4556099 
DATA_PREPARED
